# Overview
*BeamNG.tech / BeamNG.drive*

This tutorial shows you how the library interacts with custom mods. To do so we create a generic mod that runs code on the vehicle and game engine side.

In this tutorial we differentiate between mod and lua module:

mod - refers to a collection of files that change the behavior of the game (see the zip file)

lua module - refers to a lua 'library' that has a global name containing a table 


## Setup

These are the usual imports to create a simple scenario. As the point is to show the mod support, nothing is happening in the scenario. The mod only prints some text to the console, so take a look at the console or the log file you can find in the user path. Note that the console only flushes the last line to the log file if another console output line is generated.

In [1]:
import os
import zipfile
from pathlib import Path

from beamngpy import BeamNGpy, Scenario, Vehicle, set_up_simple_logging


In [2]:
USERPATH = fr'{os.getenv("LOCALAPPDATA")}/BeamNG.tech'

set_up_simple_logging()
beamng = BeamNGpy('localhost', 64256, user=USERPATH)

scenario = Scenario('smallgrid', 'On how to use custom mods')

etk = Vehicle('ego_vehicle', model='etk800', license='AI', extensions=['vehicleEngineCode'])
scenario.add_vehicle(etk, pos=(0, 0, 0), rot_quat=(0, 0, 1, 0))

pickup = Vehicle('some_vehicle', model='pickup', license='AI', extensions=['vehicleEngineCode'])
scenario.add_vehicle(pickup, pos=(5, 0, 0), rot_quat=(0, 0, 1, 0))

2022-12-05 18:59:14,648 |INFO     |beamngpy                      |Started BeamNGpy logging.


## Creating a Simple Mod

Mods can be installed by creating a zip directory in the userfolder, with default location of `%localappdata%/BeamNG.tech/<VERSION>/mods`. They have to recreate the exact same file structure as you find in the game directory in `BeamNG.tech.vX.X.X.X/lua`.

What happens here is that the two lua files in the BeamNGpy directory are zipped into a mod.

In [3]:
userpath = Path(USERPATH, '0.28')

# setting up mod
myModPath = userpath / 'mods' / 'genericResearchMod.zip' 
myModPath.parent.mkdir(parents=True, exist_ok=True)

geCode = 'gameEngineCode.lua'
zipGEpath = str(Path('lua') / 'ge' / 'extensions' / 'util' / geCode)
veCode = 'vehicleEngineCode.lua'
zipVEpath = str(Path('lua') / 'vehicle' / 'extensions' / veCode)
localDir = Path(os.path.abspath('.'))

with zipfile.ZipFile(str(myModPath), 'w') as ziph:
    ziph.write(localDir / geCode, arcname=zipGEpath)
    ziph.write(localDir / veCode, arcname=zipVEpath)

## Testing the mod

To test the mod we start BeamNG.tech and give the python BeamNG class the location of the gameengine mod within the "genericResearchMod.zip/lua/ge/extensions/" directory. This is necessary to register the file as its own lua module within the game.

After registration, it is available as util_gameEngineCode within the game - try typing `dump(util_gameEngineCode)` in the game's command prompt

In [4]:
bng = beamng.open(extensions=['util/gameEngineCode'])
scenario.make(beamng)
bng.load_scenario(scenario)
bng.start_scenario()

2022-12-05 18:59:18,383 |WARNING  |py.warnings                   |c:\dev\beamngpy\src\beamngpy\connection\connection.py:149: ResourceWarning: unclosed <socket.socket fd=1868, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 53489)>
  sleep(5)

2022-12-05 18:59:18,383 |WARNING  |py.warnings                   |c:\dev\beamngpy\src\beamngpy\connection\connection.py:149: ResourceWarning: unclosed <socket.socket fd=1868, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 53489)>
  sleep(5)

2022-12-05 18:59:18,384 |INFO     |beamngpy.BeamNGpy             |Opening BeamNGpy instance.
2022-12-05 18:59:18,391 |INFO     |beamngpy.BeamNGpy             |Started BeamNG.
2022-12-05 18:59:28,392 |INFO     |beamngpy.BeamNGpy             |Connecting to BeamNG.tech at: (localhost, 64256)
2022-12-05 18:59:29,315 |INFO     |beamngpy.BeamNGpy             |Successfully connected to BeamNG.tech.
2022-12-05 18:59:29,315 |INFO     |beamngpy

## How to Call Lua Module Functions

BeamNG.tech and BeamNGpy communicate via TCP sockets. 
Function parameters are send with the help of python dictionaries that are available as lua tables in the game. 
The mini mod from this tutorial follows the convention the BeamNGpy library established: Every value for the 'type' key helps identifying the appropriate handler, which for 'Foo' is expected to be 'handleFoo'.
The `checkMessages` function in `lua/ge/extensions/tech/techCore.lua` and `lua/vehicle/extensions/tech/techCore.lua` checks whether a corresponding function is locally available and, if not, calls the `onSocketMessage` hook for every extension. 

## Data Transfer between BeamNGpy and BeamNG
Any return value needs to be send with the help of the socket. See here an example on how to do it on the game engine side. The code for the vehicle side is the same, it just needs to be implemented in `lua/vehicle/extensions/tech/techCore.lua`.

In [5]:
def callFoo(bng):
    '''
    Executes handleFoo defined in gameEngineCode.lua.
    '''
    data = dict(type='Foo', someName='YourName')
    response = bng.connection.send(data)
    response.ack('FooAck')

def callBar(veh):
    '''
    Executes handlebar defined in vehicleEngineCode.lua.
    Here the code is executed in the VM of the etk800.
    '''
    data = dict(type='Bar', text='lorem ipsum...')
    response = veh.connection.send(data)
    response.ack('BarAck')

def dataExchange(bng):
    '''
    Demonstrates how to transfer in game data to the python side.
    '''
    data = dict(type='GetListOfVehicleModels')
    response = bng.connection.send(data).recv('ListOfVehicleModels')
    print('List of spawned vehicle models: ', response['data'])

In [6]:
callFoo(bng)

In [7]:
callBar(etk)

In [8]:
dataExchange(bng)

List of spawned vehicle models:  ['etk800', 'pickup']


### Don't see anything?

If you have trouble finding the messages in the log file or command prompt, search for 'gameEngineCode' or 'vehicleEngineCode'. These are the log tags of the respective modules.

In [9]:
bng.close()

2022-12-05 18:59:45,936 |INFO     |beamngpy.BeamNGpy             |Closing BeamNGpy instance.
2022-12-05 18:59:45,937 |INFO     |beamngpy.Vehicle              |Disconnected from vehicle some_vehicle and detached sensors.
2022-12-05 18:59:45,937 |INFO     |beamngpy.Vehicle              |Disconnected from vehicle ego_vehicle and detached sensors.
2022-12-05 18:59:45,938 |INFO     |beamngpy.BeamNGpy             |Terminating BeamNG.tech process.
